In [26]:
from sklearn.datasets import load_wine
import pandas as pd

import matplotlib.pyplot as plt 
import numpy as np

In [27]:
wine=load_wine()

df = pd.DataFrame(wine.data, columns = wine.feature_names)
df = pd.concat([df, pd.DataFrame(wine.target, columns = ["wine_class"])], axis = 1)
df.head()




,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,wine_class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


# 1. Train| test split(*)

In [28]:
from sklearn.model_selection import train_test_split

X, y = df.drop("wine_class", axis=1), df["wine_class"]
# use the same test set as in previous lecture

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42  )

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


scaler = StandardScaler()

# pipeline with StandardScaler and KNN
pipe_KNN = Pipeline([("scaler", scaler), ("knn", KNeighborsClassifier())])

# pipeline with StandardScaler and LogisticRegression
pipe_log_reg = Pipeline([("scaler", scaler), ("log", LogisticRegression(
    solver="saga", multi_class = "ovr", max_iter=10000, penalty="elasticnet"))])


param_grid_KNN = {"knn__n_neighbors": list(range(1, 50))}

l1_ratio = np.linspace(0, 1, 20)
param_grid_log_reg = {"log__l1_ratio": l1_ratio}


classifier_KNN = GridSearchCV(
    estimator=pipe_KNN, param_grid=param_grid_KNN, cv=5, scoring="accuracy")

classifier_log_reg = GridSearchCV(
    estimator=pipe_log_reg, param_grid=param_grid_log_reg, cv=5, scoring="accuracy"
)

# it will scale the data to X_train using StandardScaler
classifier_KNN.fit(X_train, y_train)

classifier_log_reg.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('log',
                                        LogisticRegression(max_iter=10000,
                                                           multi_class='ovr',
                                                           penalty='elasticnet',
                                                           solver='saga'))]),
             param_grid={'log__l1_ratio': array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])},
             scoring='accuracy')

In [30]:
df.shape
df.all




<bound method DataFrame.all of      alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0      14.23        1.71  2.43               15.6      127.0           2.80   
1      13.20        1.78  2.14               11.2      100.0           2.65   
2      13.16        2.36  2.67               18.6      101.0           2.80   
3      14.37        1.95  2.50               16.8      113.0           3.85   
4      13.24        2.59  2.87               21.0      118.0           2.80   
..       ...         ...   ...                ...        ...            ...   
173    13.71        5.65  2.45               20.5       95.0           1.68   
174    13.40        3.91  2.48               23.0      102.0           1.80   
175    13.27        4.28  2.26               20.0      120.0           1.59   
176    13.17        2.59  2.37               20.0      120.0           1.65   
177    14.13        4.10  2.74               24.5       96.0           2.05   

     flavanoids  non